In [1]:
import pandas as pd
import re
import requests
import time

In [ ]:
apple_cik = '0000320193'
header = {"User-agent":"nchasse@umich.edu"}
url = 'https://data.sec.gov/submissions/CIK' + apple_cik +'.json'
r = requests.get(url,headers=header)
data_sec = r.json()

In [ ]:
test = "https://data.sec.gov/api/xbrl/companyfacts/CIK" + apple_cik + ".json"
r = requests.get(test,headers=header)
test_sec = r.json()

In [ ]:
measures = test_sec['facts']['us-gaap'].keys()

In [ ]:
def get_request(CIK):
    header = {"User-agent":"nchasse@umich.edu"}
    url = 'https://data.sec.gov/submissions/CIK' + CIK +'.json'
    r = requests.get(url,headers=header)
    data_sec = r.json()
    return data_sec

In [ ]:
def get_measures(sec):
    return sec['facts']['us-gaap'].keys()

In [ ]:
def get_measure_keys(measures):
    measure_keys = {}
    for measure in measures:
        measure_keys = list(test_sec['facts']['us-gaap'][measure]['units'])[0]
    return measure_keys

In [ ]:
def get_data(measure,measure_units):
    quarters = []
    for dic in test_sec['facts']['us-gaap'][measure]['units'][measure_units[measure]]:
        if ('frame' in dic.keys()):
            new_dic = {}
            new_dic['end'] = dic['end']
            new_dic['frame'] = dic['frame']
            new_dic['value'] = dic['val']
            new_dic['measure'] = measure
            if re.match(r"^CY\d{4}$",dic['frame']):
                new_dic['type'] = 'Year'
            else:
                new_dic['type'] = 'Quarter'
            quarters.append(new_dic)
    df = pd.DataFrame.from_dict(quarters)
    return df

In [ ]:
def make_dataframe(measures,measure_units):
    df_final = pd.DataFrame()
    for measure in measures:
        df = get_data(measure,measure_units)
        if len(df_final) == 0:
            df_final = df
        else:
            df_final = pd.concat([df_final, df],join='outer')
    df_final = df_final.sort_values(by='end')
    return df_final

In [ ]:
#get all CIKS

In [ ]:
#Run for all CIKS and then save to Pickle File